<a id="top"></a>
# Using Updated Astrometry Solutions

<div class="alert alert-block alert-warning" style="color:black" > <b> This notebook requires creating and activating a virtual environment using the requirements file in this notebook's repository. Please also review the README file before using the notebook.</b> <br> </div>

## Table of Contents
<a id="toc"></a>

[Introduction](#intro) <br>
[Import Packages](#import) <br>

 0. [Example Data Download](#0.-Example-Data-Download)
 1. [New Extensions on FITS Files](#1.-New-extensions-on-fits-files)
 2. [Exploring different solutions](#2.-Exploring-different-solutions)
 3. [Applying a headerlet to the science extensions](#3.-Applying-a-headerlet-to-the-science-extensions)
 4. [Changing to alternate WCS solutions](#4.-Changing-to-alternate-WCS-solutions)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.1 [FIT-REL Gaia eDR3 solution](#4.1-FIT-REL-Gaia-eDR3-solution)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.2 ["a priori" solution](#4.2-"a-priori"-solution)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.3 ["distortion-only" solution](#4.3-"distortion-only"-solution)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.4 [FIT-SVM Gaia DR2 solution](#4.4-FIT-SVM-Gaia-DR2-solution)<br>
 5. [Using downloaded SVM headerlets](#5.-Using-downloaded-SVM-headerlets)
 6. [Running AstroDrizzle](#6.-Running-AstroDrizzle)

[Conclusions](#conclude) <br>
[About this Notebook](#about)

## Introduction
<a id="intro"></a>

As part of an effort to improve the absolute astrometry of HST images, STScI has created multiple new astrometric solutions for ACS and WFC3 images.  These solutions are contained in the World Coordinate System (WCS) of the images, as well as headerlet extensions and files.  This notebook provides an example workflow showing how to use different WCS solutions.

The WCS solutions are contained in the exposure file (`flt.fits` and `flc.fits`) headers. However, these updates were implemented in December 2019 as a part of HST Data Processing version 2019.5.  If data were downloaded before that, they can either be redownloaded (and will contain the new solutions) from MAST or via astroquery, or can be updated by connecting to the database containing solutions (shown below), or downloading the headerlets (also shown below).


<div class="alert alert-block alert-warning">
<b>NOTE:</b> The new solutions are used by default by the MAST pipeline! Some datasets may have certain solutions  such as fitting to Gaia eDR3, though other datasets (even in the same visit) may not! Thus, it is crucial to check which WCS solution is active. The easiest way to check this is to examine the WCSNAME keyword in the header of the SCI extensions. Alternatively, in many cases a solution may be available from the Single Visit Mosaic (SVM) products produced by MAST, which attempt to relatively align the images in a visit first, and then align them to Gaia or other external catalogs.  Getting the Single Visit Mosaic WCS solutions and applyign them to data is shown in Section 5.
</div>

For more information on Astrometry solutions, see [Section 4.5](https://hst-docs.stsci.edu/drizzpac/chapter-4-astrometric-information-in-the-header/4-5-absolute-astrometry) of the Drizzlepac Handbook. See the MAST webpage for more information on [Single Visit Mosaics (SVMs)](https://archive.stsci.edu/contents/newsletters/december-2020/hap-single-visit-mosaics-now-available).

<a id="import"></a>
## Import Packages
[Table of Contents](#toc)

***

The following Python packages are required to run the Jupyter Notebook:
 - [**os**](https://docs.python.org/3/library/os.html) - change and make directories
 - [**glob**](https://docs.python.org/3/library/glob.html) - gather lists of filenames
 - [**shutil**](https://docs.python.org/3/library/shutil.html#module-shutil) - remove directories and files
 - [**numpy**](https://numpy.org) - math and array functions
 - [**matplotlib**](https://matplotlib.org/stable/tutorials/pyplot.html) - make figures and graphics
 - [**astropy**](https://www.astropy.org) - file handling, tables, units, WCS, statistics
 - [**astroquery**](https://astroquery.readthedocs.io/en/latest/) - download data and query databases
 - [**drizzlepac**](https://www.stsci.edu/scientific-community/software/drizzlepac) - align and combine HST images

In [ ]:
import os
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.visualization import ZScaleInterval
from astroquery.mast import Observations
from stwcs.wcsutil import headerlet
from drizzlepac import astrodrizzle
from drizzlepac.processInput import getMdriztabPars
from collections import defaultdict
%matplotlib notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # Greatly improves the resolution of figures rendered in notebooks.

Some steps in this notebook require access to HST reference files, so we will create a temporary 'iref' directory for these reference files after download. This step is typically done by defining the 'iref' path in your bash profile so that all reference files for all datasets can be in one static location, but for the portability of this notebook we will create a directory. 

In [ ]:
os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = './crds_cache'
os.environ['iref'] = './crds_cache/references/hst/wfc3/'

## 0. Example Data Download

[Table of Contents](#toc)

---
MAST queries may be done using <a href="https://astroquery.readthedocs.io/en/latest/mast/mast_obsquery.html#observation-criteria-queries"> `query_criteria`</a>, where we specify: <br>

&nbsp;&nbsp;&nbsp;&nbsp;$\rightarrow$ obs_id, proposal_id, and filters 

MAST data products may be downloaded by using <a href="https://astroquery.readthedocs.io/en/latest/mast/mast_obsquery.html#downloading-data"> `download_products`</a>, where we specify:<br> 

&nbsp;&nbsp;&nbsp;&nbsp;$\rightarrow$ products = calibrated (FLT, FLC) or drizzled (DRZ, DRC) files

&nbsp;&nbsp;&nbsp;&nbsp;$\rightarrow$ type = standard products (CALxxx) or advanced products (HAP-SVM)
____

Let's find some example HST data from MAST and download it. The example used here is from visit 07 of program [14689](http://www.stsci.edu/cgi-bin/get-proposal-info?id=14689&observatory=HST). The association id7307030 contains four FLC images that have been processed by the HST WFC3 pipeline (calwf3), which includes bias subtraction, dark current correction, CTE-correction, cosmic-ray rejection, and flat-fielding. 

            id7307xfq_flc.fits
            id7307xhq_flc.fits
            id7307xlq_flc.fits
            id7307xpq_flc.fits

<div class="alert alert-block alert-info" style="color:black" >  Depending on your connection speed this cell may take a few minutes to execute. </div>

In [ ]:
obs_ids = ['IEPW14030', 'IEPW14040']

obsTable = Observations.query_criteria(obs_id=obs_ids)
products = Observations.get_product_list(obsTable)

data_prod = ['FLC', 'ASN', 'DRC'] # ['FLC', 'FLT', 'DRC', 'DRZ']
data_type = ['CALWF3'] # ['CALACS', 'CALWF3', 'CALWP2', 'HAP-SVM']

Observations.download_products(products, productSubGroupDescription=data_prod, project=data_type, cache=True)

Next, we retrieve the Hubble Advanced Product (HAP) headerlets, which we will use to change between different WCS solutions.

In [ ]:
Observations.download_products(products, productSubGroupDescription=['HLET'], project=['HAP-SVM'], cache=True)

Now to make the paths easier to work with, we move those files from their default download location into the notebook directory. In addition, we add one to the headerlet extension numbers because lists are zero indexed while the EXTVER's extensions are unity based. We do this by defining a small `correct_hdrlet_extvers()` function.

In [ ]:
for fits_file in glob.glob('./mastDownload/HST/*/*.fits'):
    fits_name = os.path.basename(fits_file)
    os.rename(fits_file, fits_name)
    
if os.path.exists('mastDownload'):
    shutil.rmtree('mastDownload')

In [ ]:
def correct_hdrlet_extvers(filename):
    """Correctly renumbers hdrlet EXTVER values"""
    hdulist = fits.open(filename, mode='update')
    hdrlet_count = 0
    for i, ext in enumerate(hdulist):
        if ext.name == 'HDRLET':
            hdrlet_count += 1
            hdulist[i].header['EXTVER'] = hdrlet_count
        else:
            continue
    hdulist.close()

In [ ]:
for flc_file in sorted(glob.glob('*flc.fits')):
    correct_hdrlet_extvers(flc_file)

Now we can check the active WCS solution in the image header. If the image is aligned to a catalog, we list the number of matches and the fit RMS converted from milliarcseconds to pixels.

In [ ]:
ext_0_keywords = ['DETECTOR', 'EXPTIME'] # extension 0 keywords.
ext_1_keywords = ['WCSNAME', 'NMATCHES', 'RMS_RA', 'RMS_DEC'] # extension 1 keywords.

# Define the detector plate scales in arcsec per pixel.
DETECTOR_SCALES = {
  'IR': 0.1283, 
  'UVIS': 0.0396, 
  'WFC': 0.05
}

formatted_data = {}
column_data = defaultdict(list)

for fits_file in sorted(glob.glob('*fl?.fits')):
    column_data['filename'].append(fits_file)
    header0 = fits.getheader(fits_file, 0)
    header1 = fits.getheader(fits_file, 1)
    
    for keyword in ext_0_keywords:
        column_data[keyword].append(header0[keyword])
    for keyword in ext_1_keywords:
        if keyword in header1:
            if 'RMS' in keyword:
                value = np.around(header1[keyword], decimals=1)
            else:
                value = header1[keyword]
            column_data[keyword].append(value)
        else:
            column_data[keyword].append(np.nan)
            
    for keyword in ['RMS_RA', 'RMS_DEC']:
        if keyword in header1:
            rms_value = header1[keyword] / 1000 / DETECTOR_SCALES[header0['DETECTOR']]
            column_data[f'{keyword}_pix'].append(np.round(rms_value, decimals=2))
        else:
            column_data[f'{keyword}_pix'].append(np.nan)

wcstable = Table(column_data)
wcstable

## 1. New extensions on FITS files

[Table of Contents](#toc)

Using `fits.info` prints basic information about the extensions in a FITS file.  In the following examples, we show operations for one file, though the same operations can be repeated in a loop for multiple files. The updated solutions should then show up as extra `HDRLET` extensions.

In [ ]:
filename = 'iepw14g4q_flc.fits'

fits.info(filename)

As seen above, there are new `HDRLET` extensions in the FITS files (as compared to the pre-2019.3 products. These extensions each contain information used to construct a World Coordinate System (WCS), which is used to transform image coordinates into physical (sky) coordinates.  Each WCS represents a uniquely derived astrometric solution.

## 2. Exploring different solutions

[Table of Contents](#toc)

Each HDRLET extension contains information describing the solution used in its creation. To investigate this we first obtain the extension numbers of the HDRLETs.

In [ ]:
ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

print(ext_indices) # To show it's consistent with the fits.info from above.

We can then loop through these extensions to see what WCS solutions are available.

In [ ]:
hdu = fits.open(filename)
print('Ext\tWCSNAME')

for ext_ind in ext_indices:
    print(ext_ind, '\t', hdu[ext_ind].header['WCSNAME'])

hdu.close()

Alternatively, we can use the `get_headerlet_kw_names()` function:

In [ ]:
new_wcsnames = headerlet.get_headerlet_kw_names(filename, kw='WCSNAME')
new_wcsnames

We can write this into a convenience function:

In [ ]:
def get_hdrlet_wcsnames(filename):
    
    """Print and return list of WCS names in HDRLET extensions of fits file"""
    
    hdu = fits.open(filename)
    ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

    print('Ext\tWCSNAME')
    new_wcsnames = []
    for ext_ind in ext_indices:
        name = hdu[ext_ind].header['WCSNAME']
        print(ext_ind, '\t', name)
        new_wcsnames.append(name)
        
    hdu.close()
    
    return new_wcsnames

In [ ]:
new_wcsnames = get_hdrlet_wcsnames(filename)

We can also see which solution is the "active" solution (the one currently applied to the SCI extensions):

In [ ]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

The nature of each solution is described here: https://drizzlepac.readthedocs.io/en/latest/mast_data_products/astrometry.html#interpreting-wcs-names. In some cases, single-visit mosaic (SVM) solution named FIT-SVM-GAIADR2 might be better than the default active solution of FIT-REL-GAIAeDR3.

## 3. Applying a headerlet to the science extensions

[Table of Contents](#toc)

To apply/activate one of the other solutions, we use the `restore_from_headerlet()` function.  This applies the WCS contained in a HDRLET extension to all SCI extensions of the image.  Doing this requires knowing which solution should be applied, which can be obtained in multiple ways. For instance, if the desired solution is `IDC_2731450pi-FIT_REL_GAIAeDR3`, we can find the `EXTVER` of the corresponding HDRLET from the list of wcs names we generated earlier.

<div class="alert alert-block alert-info">
NOTE: This is especially useful in cases where some of the exposures in a visit will have solutions that are aligned to Gaia, but others won't.  This is true for grism images in the same visit as direct images, or shallow/deep exposure combinations.
</div>

In [ ]:
# Gets the index of list element with value 'IDC_2731450pi-GSC240'.
# The index in this list + 1 is the same as the EXTVER of the corresponding HDRLET.
# We need to add 1 because lists are 0-indexed, while EXTVER's are 1 indexed.

chosen_ext = new_wcsnames.index('IDC_2731450pi-FIT_REL_GAIAeDR3')+1

In [ ]:
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)

In this case we set `archive` keyword argument to `False`.  Setting `archive` to True will preserve the currently active WCS as a new HDRLET extension on the file.  Since in our case the current solution already has a HDRLET, we do not need to archive it.  This may be useful in some cases, such as when the image has been manually aligned/transformed, and keeping a record of that solution is desired.

We can check that the solution was applied:

In [ ]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))
print(current_wcs)

We can also apply the solution via the HDRNAME:

In [ ]:
hdrlet_hdrnames = headerlet.get_headerlet_kw_names(fits.open(filename), 'HDRNAME')
desired_hdrname = hdrlet_hdrnames[new_wcsnames.index('IDC_2731450pi-GSC240')]
print(desired_hdrname)

In [ ]:
headerlet.restore_from_headerlet(filename, hdrname=desired_hdrname, archive=False, force=False)

We can also apply some logic to get the `hdrext` programatically.  For instance, if we only wanted the `IDC` (distortion calibrated) solution with the `GSC240` tag (indicating that the guide star positions had been updated), we can do the following:

In [ ]:
for i, wcsname in enumerate(new_wcsnames):
    if 'IDC' in wcsname and 'GSC240' in wcsname:
        chosen_ext = i + 1 # Add one due to 0 indexing of enumerate vs 1 indexing of EXTVER
        break

print('The desired extension is:', ('HDRLET', chosen_ext))

Finding the solution this way can be easier, as it doesn't require a full typing out of the IDCTAB name.  However, in the future, if new IDCTABs are created, there may be multiple solutions matching the criteria above, and more sophisticated logic will need to be applied.

## 4. Changing to alternate WCS solutions

[Table of Contents](#toc)

Here we look at four WCS solutions and inspect which has the best alignment with respect to stars in the HST image.

### 4.1 FIT-REL Gaia eDR3 solution

The entire association is aligned as a group with repect to Gaia when the WCS is FIT-REL. 

In [ ]:
chosen_ext = new_wcsnames.index('IDC_2731450pi-FIT_REL_GAIAeDR3') + 1
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))
print(current_wcs)

If the original solution is desired, it can be restored using the methods shown above, by replacing the WCSNAME simply with `IDC_2731450pi`, or whatever the name of the IDCTAB for that file is.  To get that name, the following procedure can be done:


### 4.2 "a priori" solution

In [ ]:
chosen_ext = new_wcsnames.index('IDC_2731450pi-GSC240') + 1
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))
print(current_wcs)

### 4.3 "distortion only" solution

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Data taken after October 2017 may not have a solution of the form IDC_xxxxxxxxx, as the pointing information of the telescope was already calculated using GSC 2.4.0.  As such, the "old" solution may be of the same form as the a priori solution, i.e.: IDC_xxxxxxxxx-GSC240.
</div>

In [ ]:
chosen_ext = new_wcsnames.index('IDC_2731450pi') + 1
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))
print(current_wcs)

## 5. Using downloaded SVM headerlets

[Table of Contents](#toc)

In cases like the example provided here, images from the same visit may have different WCS solution types (i.e. some are `FIT-REL-GAIAeDR3` and others are `GSC240`).  This may lead to images not being aligned, especially if some images use a Gaia based solutions and others don't.  However, we can apply the solutions used in the SVM images, which are derived from first relatively aligning the data to each other, and then aligning the group of images to an absolute reference catalog.  Thus, they are often a better solution for datasets with a variety of filters/depths.

In [ ]:
hlet_files = sorted(glob.glob('*hlet.fits'))

Let's look at WCS solutions in the headerlet for each image

In [ ]:
root_to_hlet_dict = {}
for hlet in hlet_files:
    dest_image = fits.getval(hlet, 'DESTIM')
    root_to_hlet_dict[dest_image] = hlet
    print(hlet, dest_image, fits.getval(hlet, 'WCSNAME', 1))

Now we simply have to match each headerlet to its corresponding flc file, and apply it.

In [ ]:
for flc in sorted(glob.glob('*flc.fits')):
    root = fits.getval(flc, 'rootname')
    headerlet.apply_headerlet_as_primary(flc, hdrlet=root_to_hlet_dict[root], attach=True)

## 6. Running `AstroDrizzle`

[Table of Contents](#toc)

Because the drizzling process is directly affected by the WCS's of the input images, the WCS of the drizzled image cannot be changed as simply as shown above for FLC images.  To use an astrometric solution (other than the one  applied to the FLT/FLC at the time of drizzling), the images will have to be re-drizzled after activating the desired WCS in the FLT/FLC images. Here we want to set the WCS solution for all of the input FLC files to be the FIT-SVM-GAIAeDR3 solution (see section 5).

<div class="alert alert-block alert-warning">
<b>NOTE:</b> The images input to astrodrizzle should use the same WCS solution, or the drizzling process will produce poor results.
</div>

In [ ]:
asn_dict = defaultdict(list)

for flc in sorted(glob.glob('*flc.fits')):
    asn_id = fits.getval(flc, 'asn_id')
    if asn_id == 'NONE':
        asn_id = fits.getval(flc, 'rootname')
    asn_id = asn_id.lower()
    asn_dict[asn_id].append(flc)
asn_dict

Here we get some recommended values for drizzling from the MDRIZTAB reference file.  The parameters in this file are different for each detector and are based on the number of input frames. These are a good starting point for drizzling and may be adjusted accordingly.

In [ ]:
input_images = sorted(glob.glob('*flc.fits'))
mdz = fits.getval(input_images[0], 'MDRIZTAB', ext=0).split('$')[1]
print('Searching for the MDRIZTAB file:', mdz)
get_mdriztab = os.system('crds sync --hst --files '+mdz+' --output-dir '+os.environ['iref'])

In [ ]:
def get_vals_from_mdriztab(input_images, kw_list=['driz_sep_bits', 
                                                  'combine_type', 
                                                  'driz_cr_snr', 
                                                  'driz_cr_scale', 
                                                  'final_bits']):
    
    '''Get only selected parameters from the MDRIZTAB.'''
    mdriz_dict = getMdriztabPars(input_images)
    
    requested_params = {}
    
    print('Outputting the following parameters:')
    for k in kw_list:
        requested_params[k] = mdriz_dict[k]
        print(k, mdriz_dict[k])
    
    return requested_params

In [ ]:
selected_params = get_vals_from_mdriztab(asn_dict['iepw14030'])

In [ ]:
for asn_id in asn_dict:
    
    input_images = asn_dict[asn_id]
    
    # To override any of the above values:
    # selected_params['driz_sep_bits'] = '64, 32'
    # selected_params['final_bits']    = '64, 32'
    # selected_params['combine_type']  = 'minmed'
    # selected_params['driz_cr_snr']   = '4.0 3.5'
    # selected_params['driz_cr_scale'] = '1.5 1.2'

    selected_params = get_vals_from_mdriztab(input_images)
    
    astrodrizzle.AstroDrizzle(input_images, 
                              output=f'{asn_id}_updated_wcs',
                              preserve=False,
                              clean=True, 
                              build=True,
                              context=False,
                              skymethod='match',
                              in_memory=True,
                              **selected_params)
clear_output()

First we will display the default pipeline DRC image retrieved from MAST to show the astrometric offset. We define the center and scaling to be the same for both sets.

In [ ]:
center = [10.0095776, 40.5014080]
z = ZScaleInterval()

In [ ]:
# Display the two science images and zoom in on an object to see the astrometric error.
image1 = 'iepw14030_drc.fits'
image2 = 'iepw14040_drc.fits'

sci_image1 = fits.getdata(image1)
sci_image2 = fits.getdata(image2)
wcs_image1 = WCS(fits.getheader(image1, 1))
wcs_image2 = WCS(fits.getheader(image2, 1))
x1, y1 = wcs_image1.world_to_pixel_values([center])[0].astype(int)
x2, y2 = wcs_image2.world_to_pixel_values([center])[0].astype(int)

fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(1, 2, 1, projection=wcs_image1)
ax2 = fig.add_subplot(1, 2, 2, projection=wcs_image2)

ax1.set_title('WCS: '+fits.getval(image1, 'WCSNAME', ext=('SCI', 1)))
ax2.set_title('WCS: '+fits.getval(image2, 'WCSNAME', ext=('SCI', 1)))
ax1.imshow(sci_image1, vmin=z.get_limits(sci_image1)[0], vmax=z.get_limits(sci_image1)[1]*5, cmap='Greys_r', origin='lower', interpolation='none')
ax2.imshow(sci_image2, vmin=z.get_limits(sci_image2)[0], vmax=z.get_limits(sci_image2)[1]*5, cmap='Greys_r', origin='lower', interpolation='none')

ax1.set_xlim(x1-50, x1+50)
ax1.set_ylim(y1-50, y1+50)
ax2.set_xlim(x2-50, x2+50)
ax2.set_ylim(y2-50, y2+50)
ax1.grid(lw=1, color='white', ls=':')
ax2.grid(lw=1, color='white', ls=':')
plt.tight_layout()

Then we will display the redrizzled image which used the FIT-SVM-GAIAeDR3 WCS, which restores the alignment.

In [ ]:
# Display the two science images and zoom in on an object to see the astrometric error.
image1 = 'iepw14030_updated_wcs_drc.fits'
image2 = 'iepw14040_updated_wcs_drc.fits'

sci_image1 = fits.getdata(image1)
sci_image2 = fits.getdata(image2)
wcs_image1 = WCS(fits.getheader(image1, 1))
wcs_image2 = WCS(fits.getheader(image2, 1))
x1, y1 = wcs_image1.world_to_pixel_values([center])[0].astype(int)
x2, y2 = wcs_image2.world_to_pixel_values([center])[0].astype(int)

fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(1, 2, 1, projection=wcs_image1)
ax2 = fig.add_subplot(1, 2, 2, projection=wcs_image2)

ax1.set_title('WCS: '+fits.getval(image1, 'WCSNAME', ext=('SCI', 1)))
ax2.set_title('WCS: '+fits.getval(image2, 'WCSNAME', ext=('SCI', 1)))
ax1.imshow(sci_image1, vmin=z.get_limits(sci_image1)[0], vmax=z.get_limits(sci_image1)[1]*5, cmap='Greys_r', origin='lower', interpolation='none')
ax2.imshow(sci_image2, vmin=z.get_limits(sci_image2)[0], vmax=z.get_limits(sci_image2)[1]*5, cmap='Greys_r', origin='lower', interpolation='none')

ax1.set_xlim(x1-50, x1+50)
ax1.set_ylim(y1-50, y1+50)
ax2.set_xlim(x2-50, x2+50)
ax2.set_ylim(y2-50, y2+50)
ax1.grid(lw=1, color='white', ls=':')
ax2.grid(lw=1, color='white', ls=':')
plt.tight_layout()

<a id="conclude"></a>
## Conclusions

[Table of Contents](#toc)

This notebook demonstrates how to access and apply different WCS solutions from exposure and SVM headerlets. In general, it is always preferred to have consistent WCS solutions across exposures, especially from the same visit. Users can also custom align their exposures to one another, as well as to external catalogs such as SDSS and Gaia. This process is detailed in the [align_to_catalogs](https://github.com/spacetelescope/hst_notebooks/blob/main/notebooks/DrizzlePac/align_to_catalogs/align_to_catalogs.ipynb) notebook.

<a id="about"></a>
## About this Notebook
    
    Created: 14 Dec 2018;     V. Bajaj
    Updated: 23 May 2024;     M. Revalski, V. Bajaj, & J. Mack

**Source:** GitHub [spacetelescope/hst_notebooks](https://github.com/spacetelescope/hst_notebooks)

<a id="add"></a>
## Additional Resources

Below are some additional resources that may be helpful. Please send any questions through the [HST Help Desk](https://stsci.service-now.com/hst), selecting the DrizzlePac category.

- [WFC3 Website](https://www.stsci.edu/hst/instrumentation/wfc3)
- [WFC3 Data Handbook](https://hst-docs.stsci.edu/wfc3dhb)
- [WFC3 Instrument Handbook](https://hst-docs.stsci.edu/wfc3ihb)

<a id="cite"></a>
## Citations
If you use Python packages such as `astropy`, `astroquery`, `drizzlepac`, `matplotlib`, or `numpy` for published research, please cite the authors.

Follow these links for more information about citing various packages:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)
* [Citing `drizzlepac`](https://zenodo.org/records/3743274)
* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/citing.html)
* [Citing `numpy`](https://numpy.org/citing-numpy/)
***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 